# Modelo lineal del artículo

En este modelo que es de la siguiente forma:

necesitaremos la base de datos que contruimos con toda la información relevante de los equipos!

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from matplotlib.colors import ListedColormap
from sklearn.naive_bayes import GaussianNB
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
import pandas as pd
import numpy as np
import array
import math
import os
import warnings
print('Modulos importados')

Modulos importados


In [2]:
# Configuraciones
warnings.filterwarnings('ignore')
# Reduzcamos el número de línea a leer
pd.options.display.max_rows = 15

In [3]:
# Veamos el directorio actual de trabajo
print(os.getcwd())
# El directorio anterior es el correcto, pero si no lo fuese, hacemos lo sigueinte:
path = '/home/usuario/Documentos/Github/Proyectos/MLB_HN'
os.chdir(path)

/home/usuario/Documentos/Github/Proyectos/MLB_HN/Models/Linear_models/Free_agent


## Agentes libres

Leamos los archivos correspondientes

In [4]:
# Años de análisis
period = 11
csv = '.csv'
# Directorios
names_path = 'Data/Teams/team_acronym'
teams_path = 'Data/New_Data/Teams/free_agents_team_'
batters_path = 'Data/New_Data/Hitters/Free_Agent/free_agents_batters_'
pitchers_path = 'Data/New_Data/Pitchers/Free_Agent/free_agents_pitchers_'
# Originales:
df_team = [None]*period
df_team_copy = [None]*period
df_batters = [None]*period
df_batters_copy = [None]*period
df_pitchers = [None]*period
df_pitchers_copy = [None]*period
df_players = [None]*period
df_players_copy = [None]*period

Keamos todos las bases de datos correspondientes

In [5]:
names = pd.read_csv(names_path + csv)
for year in range(0,period):
    df_batters[year] = pd.read_csv(batters_path + str(2011 + year) + csv)
    df_batters_copy[year] = df_batters[year].copy()
    
    df_pitchers[year] = pd.read_csv(pitchers_path + str(2011 + year) + csv)
    df_pitchers_copy[year] = df_pitchers[year].copy()
    
    df_team[year] = pd.read_csv(teams_path + str(2011 + year) + csv)
    df_team_copy[year] = df_team[year].copy()

Observemos el contenido de las bases de datos

In [6]:
# Nombres de todos los equipos en todss las bases de datos
names.head()

,Equipo,Acronimo,Estado
0,Arizona Diamondbacks,ARI,Arizona
1,Atlanta Braves,ATL,Georgia
2,Baltimore Orioles,BAL,Maryland
3,Boston Red Sox,BOS,Massachusetts
4,Chicago Cubs,CHC,Illinois


In [7]:
# Equipos
df_team_copy[1].head()

,Equipo,Cantidad_agentes_libres,Valor_contrato,Acronimo,Victorias,Juegos totales,Promedio_victorias
0,Los Angeles Angels,4,321150000,LAA,89,162,0.549383
1,Detroit Tigers,3,221000000,DET,88,162,0.543210
2,Miami Marlins,6,203300000,MIA,69,162,0.425926
3,Philadelphia Phillies,7,57650000,PHI,81,162,0.500000
4,Los Angeles Dodgers,9,44651311,LAD,86,162,0.530864


In [8]:
# Bateadores
df_batters_copy[1].head()

,Jugador,Valor_contrato,Valor_promedio_contrato,Posicion,Juegos,Porcetnaje_juegos,At-bats,Bateos,Home-runs,RBI,Porcentaje_bateo,OPS,Equipo,Sueldo
0,Albert Pujols,240000000,24000000,DH,154,0.951,607,173,30,105,0.285,0.859,LAA,12000000
1,Prince Fielder,214000000,23777778,DH,162,1.000,581,182,30,108,0.313,0.940,DET,23150000
2,Jose Reyes,106000000,17666667,3B,160,0.988,642,184,11,57,0.287,0.780,MIA,10000000
3,C.J. Wilson,77500000,15500000,SP,34,0.210,5,0,0,0,0.000,0.167,LAA,10550000
4,Mark Buehrle,58000000,14500000,SP,31,0.191,67,3,0,1,0.045,0.104,MIA,7000000


In [9]:
# PItchers
df_pitchers_copy[1].head()

,Jugador,Valor_contrato,Valor_promedio_contrato,Posicion,Juegos,Juegos_iniciados,Inning_pitched,Bateos_pitcher,Carreras,Carreras_ganadas,Walks,Strike-outs,Wins,Losses,Saves,WHIP,ERA,Equipo,Sueldo
0,C.J. Wilson,77500000,15500000,SP,34,34,202.3,181,102,86,91,173,13,10,0,1.34,3.83,LAA,10550000
1,Mark Buehrle,58000000,14500000,SP,31,31,202.3,197,88,84,40,125,13,13,0,1.17,3.74,MIA,7000000
2,Jonathan Papelbon,50000000,12500000,RP,70,0,70.0,56,22,19,18,92,5,6,38,1.06,2.44,PHI,11000058
3,Heath Bell,27000000,9000000,RP,73,0,63.7,70,38,36,29,59,4,5,19,1.55,5.09,MIA,7000000
4,Joe Nathan,14000000,7000000,RP,66,0,64.3,55,23,20,13,78,3,5,37,1.06,2.80,TEX,8050000


### Construcción de la base de datos para el modelo

#### Base de datos de los equipos

Primero, tenemos que determinar cuál es el mínimo y máximo del promedio de victorias de cada equipo a lo largo del periodo de análisis. Primero, creemos un diccionario que guarde en una lista ambos valores cuya llave asociada sea el nombre del equipo. Se tomará el promedio y no el número de victorias puesto que no todos los equipos juegan la misma cantidad de partidos en una temporada y habría fallos en el análisis en años como el 2020 por la pandemia que provocó que la cantidad de juegos por temporada fuiera mucho menor  a la usual, siendo alrededor de 60 juegos en dicha temporada.

Para crear este diccionario usaremos la base de datos de los acrónimos puesto que contiene todos los equipos que puedan haber en las bases de datos de los equipos en el periodo de análisis

In [10]:
minmax_victories = {}
length_names = len(names['Equipo'])
# CReemos las llaves con los nombres de los equipos
for i in range(0,length_names):
    minmax_victories[str(names['Equipo'].iloc[i])] = [None]*2

Como valores de comparación para las victorias usaremos cualquier base de datos que contenga la misma cantidad de filas que la base de datos *names*. Obtengamos el índice de las bases de datos que cumplan esa condición.

In [11]:
for year in range(0,period):
    if len(df_team_copy[year]['Equipo']) == length_names:
        print(year)

2
3
4
5
6
8
10


In [12]:
# Usaremos la tercera base de datos
team_name = list(minmax_victories.keys())
for i in range(length_names):
    for j in range(length_names):
        if team_name[i] == df_team_copy[2]['Equipo'].iloc[j]:
            minmax_victories[team_name[i]][0] = df_team_copy[2]['Promedio_victorias'].iloc[j]
            minmax_victories[team_name[i]][1] = df_team_copy[2]['Promedio_victorias'].iloc[j]

Ahora, recorreremos el promedio de victorias de todas las bases de datos para hallar el menor y mayor número de victorias que ha tenido cada equipo en el periodo de análisis.

In [13]:
for year in range(period):
    # Nombres de los equipos en la base de datos
    team_year_name = list(df_team_copy[year]['Equipo'])
    
    for i in range(length_names):
        # Mínimo actual
        min_victory = minmax_victories[team_name[i]][0]
        # Máximo actual:
        max_victory = minmax_victories[team_name[i]][1]
        
        for j in range(len(team_year_name)):
            
            if team_name[i] == team_year_name[j]:
                year_victory = df_team_copy[year]['Promedio_victorias'].iloc[j]
                
                # Mínimo nuevo:
                if year_victory < min_victory:
                    minmax_victories[team_name[i]][0] = year_victory
                # Máximo nuevo:
                elif year_victory > max_victory:
                    minmax_victories[team_name[i]][1] = year_victory

Los valores de la lista ahora son de la siguiente forma:

In [14]:
print(minmax_victories[team_name[1]])

[0.40625, 0.6024844720496895]


A continuación, se creará una columna que indique si la victoria de la temporada -correspondiente a la base de datos donde se situe- está más cercana del promedio más alto de victorias o del menor. 1 indicará que está más cercano a la mayor victoria, y 0 en otro caso -en caso de empate, se eligirá al 1.

In [15]:
for year in range(period):
    # Nombres de los equipos en la base de datos
    team_year_name = list(df_team_copy[year]['Equipo'])
    # COlumna dummy:
    df_team_copy[year]['I_t'] = -1
    
    for i in range(length_names):
        # Mínimo actual
        min_victory = minmax_victories[team_name[i]][0]
        # Máximo actual:
        max_victory = minmax_victories[team_name[i]][1]
        
        for j in range(len(team_year_name)):
            
            if team_name[i] == team_year_name[j]:
                year_victory = df_team_copy[year]['Promedio_victorias'].iloc[j]
                
                # Distancia al mínimo:
                dist_min = year_victory - min_victory
                # Distancia al máximo:
                dist_max = max_victory - year_victory
                
                # Mínimo nuevo:
                if dist_min < dist_max:
                    df_team_copy[year]['I_t'].iloc[j] = 1
                # Máximo nuevo
                else:  
                    df_team_copy[year]['I_t'].iloc[j] = 0

In [16]:
df_team_copy[6].tail()

,Equipo,Cantidad_agentes_libres,Valor_contrato,Acronimo,Victorias,Juegos totales,Promedio_victorias,I_t
24,Milwaukee Brewers,3,7550000,MIL,86,162,0.530864,0
25,Chicago White Sox,1,6000000,CHW,67,162,0.413580,1
26,Boston Red Sox,1,5500000,BOS,93,162,0.574074,0
27,Cincinnati Reds,2,5300000,CIN,68,162,0.419753,1
28,Detroit Tigers,1,2000000,DET,64,162,0.395062,1


Usemos la variable dummy para crear la columna que contenga el siguiente dato:

$$
X = \left( -1 \right)^{I_{t}^{-}}\frac{y_{t}}{\sqrt{y_H}}
$$

In [17]:
for year in range(period):
    # Nombres de los equipos en la base de datos
    team_year_name = list(df_team_copy[year]['Equipo'])
    # Columna de la nueva variable
    df_team_copy[year]['X'] = -1
    
    for i in range(length_names):        
        for j in range(len(team_year_name)):
            
            if team_name[i] == team_year_name[j]:
                # Variables auxiliares
                mean_victory = df_team_copy[year]['Promedio_victorias'].iloc[j]
                dummy = df_team_copy[year]['I_t'].iloc[j]
                s_max_victory = np.power(minmax_victories[team_name[i]][1],0.5)
                if dummy == 0:
                    parity = 1
                else:
                    parity = -1
                
                # Variable dummy nueva
                df_team_copy[year]['X'].iloc[j] = parity*mean_victory/s_max_victory

In [18]:
df_team_copy[1].tail()

,Equipo,Cantidad_agentes_libres,Valor_contrato,Acronimo,Victorias,Juegos totales,Promedio_victorias,I_t,X
21,Kansas City Royals,2,5000000,KC,72,162,0.444444,1,-0.581656
22,San Francisco Giants,3,4800000,SF,94,162,0.580247,0,0.712844
23,Atlanta Braves,1,1000000,ATL,94,162,0.580247,0,0.747549
24,Chicago White Sox,1,900000,CHW,85,162,0.524691,0,0.686982
25,Houston Astros,1,750000,HOU,55,162,0.339506,1,-0.418415


In [19]:
df_team_copy[1].tail()

,Equipo,Cantidad_agentes_libres,Valor_contrato,Acronimo,Victorias,Juegos totales,Promedio_victorias,I_t,X
21,Kansas City Royals,2,5000000,KC,72,162,0.444444,1,-0.581656
22,San Francisco Giants,3,4800000,SF,94,162,0.580247,0,0.712844
23,Atlanta Braves,1,1000000,ATL,94,162,0.580247,0,0.747549
24,Chicago White Sox,1,900000,CHW,85,162,0.524691,0,0.686982
25,Houston Astros,1,750000,HOU,55,162,0.339506,1,-0.418415


Por último, solo nos quedaremos con las columnas de utilidad que son el acrónimo del equipo y la variable $X$.

In [20]:
for year in range(0,period):
    df_team_copy[year] = df_team_copy[year][["Acronimo", "X"]]

In [21]:
df_team_copy[1].head()

,Acronimo,X
0,LAA,0.704166
1,DET,0.716941
2,MIA,-0.592555
3,PHI,0.702782
4,LAD,-0.629194


### Batters

Preparemos las bases de datos de los *batters*.

Por otro lado, tenemos que traer los valores de los salarios -en dolares- a valor presente, del 2021, ajustado por la inflación. Usaremos la página [CPI Inflation Calculator](https://www.in2013dollars.com/)

Guardaremos los porcentajes de cambio de cada año en un diccionario, de paso, cambiaremos el nombre de la columna *Equipo* por *Acrónimo*.

In [22]:
change_inflation_percentage = {2011:20.46, 2012:18.02, 2013:16.32, 2014:14.46, 2015:14.33, 2016:12.90, 2017:10.55, 2018:7.86, 2019:5.99, 2020:4.70, 2021:1}

In [23]:
for year in range(0,period):
    df_batters_copy[year][["Salario_ajustado_2021"]] = df_batters_copy[year][["Sueldo"]]*(1 + change_inflation_percentage[2011 + year]*0.01)
    df_batters_copy[year].rename(columns = {'Equipo':'Acronimo'}, inplace = True)

In [24]:
df_batters_copy[1].head()

,Jugador,Valor_contrato,Valor_promedio_contrato,Posicion,Juegos,Porcetnaje_juegos,At-bats,Bateos,Home-runs,RBI,Porcentaje_bateo,OPS,Acronimo,Sueldo,Salario_ajustado_2021
0,Albert Pujols,240000000,24000000,DH,154,0.951,607,173,30,105,0.285,0.859,LAA,12000000,14162400.0
1,Prince Fielder,214000000,23777778,DH,162,1.000,581,182,30,108,0.313,0.940,DET,23150000,27321630.0
2,Jose Reyes,106000000,17666667,3B,160,0.988,642,184,11,57,0.287,0.780,MIA,10000000,11802000.0
3,C.J. Wilson,77500000,15500000,SP,34,0.210,5,0,0,0,0.000,0.167,LAA,10550000,12451110.0
4,Mark Buehrle,58000000,14500000,SP,31,0.191,67,3,0,1,0.045,0.104,MIA,7000000,8261400.0


Lo que se tiene que hacer a continuación es añadir la variable $X$ a la base de datos anterior, esto se tiene que hacer de acuerdo al equipo

In [25]:
for year in range(0,period):
    df_batters_copy[year] = pd.merge(df_batters_copy[year], df_team_copy[year], on = 'Acronimo')

In [26]:
df_batters_copy[1].head()

,Jugador,Valor_contrato,Valor_promedio_contrato,Posicion,Juegos,Porcetnaje_juegos,At-bats,Bateos,Home-runs,RBI,Porcentaje_bateo,OPS,Acronimo,Sueldo,Salario_ajustado_2021,X
0,Albert Pujols,240000000,24000000,DH,154,0.951,607,173,30,105,0.285,0.859,LAA,12000000,1.416240e+07,0.704166
1,C.J. Wilson,77500000,15500000,SP,34,0.210,5,0,0,0,0.000,0.167,LAA,10550000,1.245111e+07,0.704166
2,Bobby Abreu,401311,401311,LF,200,0.617,438,106,6,48,0.242,0.693,LAA,8598689,1.014817e+07,0.704166
3,Prince Fielder,214000000,23777778,DH,162,1.000,581,182,30,108,0.313,0.940,DET,23150000,2.732163e+07,0.716941
4,Omar Infante,8000000,4000000,2B,149,0.920,554,152,12,53,0.274,0.719,DET,2142076,2.528078e+06,0.716941


Solo interesan las columnas que contienen el nombre del jugador, su salario ajustado y la variable $X$.

In [27]:
for year in range(0,period):
    df_batters_copy[year] = df_batters_copy[year][["Jugador", "Salario_ajustado_2021", "X"]]

In [28]:
df_batters_copy[1].head()

,Jugador,Salario_ajustado_2021,X
0,Albert Pujols,1.416240e+07,0.704166
1,C.J. Wilson,1.245111e+07,0.704166
2,Bobby Abreu,1.014817e+07,0.704166
3,Prince Fielder,2.732163e+07,0.716941
4,Omar Infante,2.528078e+06,0.716941


### Pitchers

Repetiremos el mismo proceso que con las bases de datos de los *batters*

In [29]:
for year in range(0,period):
    df_pitchers_copy[year][["Salario_ajustado_2021"]] = df_pitchers_copy[year][["Sueldo"]]*(1 + change_inflation_percentage[2011 + year]*0.01)
    df_pitchers_copy[year].rename(columns = {'Equipo':'Acronimo'}, inplace = True)
    
    df_pitchers_copy[year] = pd.merge(df_pitchers_copy[year], df_team_copy[year], on = 'Acronimo')
    
    df_pitchers_copy[year] = df_pitchers_copy[year][["Jugador", "Salario_ajustado_2021", "X"]]

In [30]:
df_pitchers_copy[1].head()

,Jugador,Salario_ajustado_2021,X
0,C.J. Wilson,1.245111e+07,0.704166
1,LaTroy Hawkins,3.540600e+06,0.704166
2,Mark Buehrle,8.261400e+06,-0.592555
3,Heath Bell,8.261400e+06,-0.592555
4,Jonathan Papelbon,1.298227e+07,0.702782


### Players

Debido a que hay agentes libres que son tanto *batters* como *pitchers*, haremos el merge por año.

Sin embargo, primero tenemos que obtener como referencia el número de jugadores que tendrá dicho dataframe; esto, para evitar errores. Para ello, necesitamos obtener los nombres de cada base de datos por año y unirlos ya sea eliminando duplicados o con otro método. Finalmente, se imprimirá la longitud **mínima** de dicha lista. Se dice mínima puesto que aquí no estamos haciendo distinción por caracetrísticas ajenas al nombre.

In [31]:
for year in range(0,period):
    print("Año: " + str(2011 + year))
    print("Número de nombres únicos en batters: ")
    batters_names = np.unique(df_batters_copy[year]["Jugador"].tolist())
    print(len(batters_names))
    
    print("Número de nombres únicos en pitchers: ")
    pitchers_names = np.unique(df_pitchers_copy[year]["Jugador"].tolist())
    print(len(pitchers_names))
    
    print("Número de nombres únicos en el merge:")
    merge_players = np.unique(batters_names.tolist() + pitchers_names.tolist())
    print(len(merge_players))
    print()

Año: 2011
Número de nombres únicos en batters: 
1
Número de nombres únicos en pitchers: 
0
Número de nombres únicos en el merge:
1

Año: 2012
Número de nombres únicos en batters: 
45
Número de nombres únicos en pitchers: 
18
Número de nombres únicos en el merge:
52

Año: 2013
Número de nombres únicos en batters: 
78
Número de nombres únicos en pitchers: 
51
Número de nombres únicos en el merge:
101

Año: 2014
Número de nombres únicos en batters: 
89
Número de nombres únicos en pitchers: 
67
Número de nombres únicos en el merge:
122

Año: 2015
Número de nombres únicos en batters: 
76
Número de nombres únicos en pitchers: 
68
Número de nombres únicos en el merge:
110

Año: 2016
Número de nombres únicos en batters: 
101
Número de nombres únicos en pitchers: 
72
Número de nombres únicos en el merge:
130

Año: 2017
Número de nombres únicos en batters: 
64
Número de nombres únicos en pitchers: 
41
Número de nombres únicos en el merge:
80

Año: 2018
Número de nombres únicos en batters: 
58
Nú

Para facilitar el merge de las bases de datos eliminaremos los registros repetidos. Esto tiene como objetivo usar el método concat para evitarnos problemas de duplicados de columnas

In [32]:
for year in range(0,period):
    batters_names = np.unique(df_batters_copy[year]["Jugador"].tolist())
    pitchers_names = np.unique(df_pitchers_copy[year]["Jugador"].tolist())
    
    df_players_copy[year] = pd.concat([df_batters_copy[year], df_pitchers_copy[year]]).drop_duplicates().reset_index(drop=True)
    print("Año " + str(2011 + year) + ":")
    print(str(len(df_players_copy[year]["Jugador"])) + "\n")

Año 2011:
1

Año 2012:
54

Año 2013:
101

Año 2014:
123

Año 2015:
112

Año 2016:
131

Año 2017:
80

Año 2018:
86

Año 2019:
87

Año 2020:
99

Año 2021:
105



In [33]:
df_players_copy[1].head()

,Jugador,Salario_ajustado_2021,X
0,Albert Pujols,1.416240e+07,0.704166
1,C.J. Wilson,1.245111e+07,0.704166
2,Bobby Abreu,1.014817e+07,0.704166
3,Prince Fielder,2.732163e+07,0.716941
4,Omar Infante,2.528078e+06,0.716941


Cuando hagamos el merge, tendremos que eliminar los duplicados restantes que no se pudieron hacer enlos dos códigos anteriores debido al hecho de que solo eliminamos duplicados exactos, pero ahora lo haremos específicamente con los nombres.

In [34]:
panel_data = pd.merge(df_players_copy[0], df_players_copy[1], how = 'outer', on = 'Jugador')

for year in range(2,period):
    panel_data = pd.merge(panel_data, df_players_copy[year], how = 'outer', on = 'Jugador')

panel_data.drop_duplicates(subset = 'Jugador', inplace = True)

In [35]:
panel_data.head()

,Jugador,Salario_ajustado_2021_x,X_x,Salario_ajustado_2021_y,X_y,Salario_ajustado_2021_x,X_x,Salario_ajustado_2021_y,X_y,Salario_ajustado_2021_x,...,Salario_ajustado_2021_x,X_x,Salario_ajustado_2021_y,X_y,Salario_ajustado_2021_x,X_x,Salario_ajustado_2021_y,X_y,Salario_ajustado_2021,X
0,Wil Nieves,933565.0,0.7698,9.441600e+05,0.659912,930560.0,0.659912,1287675.0,0.633371,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Albert Pujols,NaN,NaN,1.416240e+07,0.704166,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,576205.0,0.775519
2,C.J. Wilson,NaN,NaN,1.245111e+07,0.704166,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Bobby Abreu,NaN,NaN,1.014817e+07,0.704166,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Prince Fielder,NaN,NaN,2.732163e+07,0.716941,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
panel_data.tail()

,Jugador,Salario_ajustado_2021_x,X_x,Salario_ajustado_2021_y,X_y,Salario_ajustado_2021_x,X_x,Salario_ajustado_2021_y,X_y,Salario_ajustado_2021_x,...,Salario_ajustado_2021_x,X_x,Salario_ajustado_2021_y,X_y,Salario_ajustado_2021_x,X_x,Salario_ajustado_2021_y,X_y,Salario_ajustado_2021,X
622,Alex Colome,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5050000.0,-0.567195
623,Anthony Bass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1010000.0,-0.575380
624,Andrew Chafin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2777500.0,0.685343
625,Keone Kela,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1212000.0,-0.617853
626,Chris Devenski,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1010000.0,-0.429145


In [37]:
panel_data.shape

(609, 23)

Se aprecia que necesitamos reiniciar el índice, pero primero lo ordenaremos alfabéticamente por motivos de *buenas prácticas*.

In [38]:
panel_data = panel_data.sort_values(by = 'Jugador', ascending = True)

In [39]:
panel_data.reset_index(drop = True, inplace = True)
panel_data.head()

,Jugador,Salario_ajustado_2021_x,X_x,Salario_ajustado_2021_y,X_y,Salario_ajustado_2021_x,X_x,Salario_ajustado_2021_y,X_y,Salario_ajustado_2021_x,...,Salario_ajustado_2021_x,X_x,Salario_ajustado_2021_y,X_y,Salario_ajustado_2021_x,X_x,Salario_ajustado_2021_y,X_y,Salario_ajustado_2021,X
0,A.J. Burnett,NaN,NaN,NaN,NaN,NaN,NaN,1.287675e+07,0.633371,9718050.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A.J. Ellis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2763750.0,0.661257,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A.J. Griffin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A.J. Pierzynski,NaN,NaN,NaN,NaN,8724000.0,0.73127,1.917651e+05,-0.699237,2743920.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A.J. Pollock,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4239600.0,0.772974,NaN,NaN,NaN,NaN


In [40]:
panel_data.tail()

,Jugador,Salario_ajustado_2021_x,X_x,Salario_ajustado_2021_y,X_y,Salario_ajustado_2021_x,X_x,Salario_ajustado_2021_y,X_y,Salario_ajustado_2021_x,...,Salario_ajustado_2021_x,X_x,Salario_ajustado_2021_y,X_y,Salario_ajustado_2021_x,X_x,Salario_ajustado_2021_y,X_y,Salario_ajustado_2021,X
604,Zach Putnam,NaN,NaN,NaN,NaN,NaN,NaN,572300.0,-0.589997,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
605,Zack Britton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,13778700.0,0.797372,NaN,NaN,NaN,NaN
606,Zack Cozart,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.366227e+07,-0.632958,NaN,NaN,NaN,NaN,NaN,NaN
607,Zack Greinke,NaN,NaN,NaN,NaN,22100800.0,-0.673092,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,Zack Wheeler,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,22510500.0,0.65593,NaN,NaN


Por otro lado, solo resta cambiar el nombre de las columnas u ordenar las columnas de acuerdo a lo que necesitamos.

In [41]:
panel_data.columns = ['Jugador', 'Salario_2011', 'X_2011', 'Salario_2012', 'X_2012', 'Salario_2013', 'X_2013', 'Salario_2014', 'X_2014'
                     , 'Salario_2015', 'X_2015', 'Salario_2016', 'X_2016', 'Salario_2017', 'X_2017', 'Salario_2018', 'X_2018'
                     , 'Salario_2019', 'X_2019', 'Salario_2020', 'X_2020', 'Salario_2021', 'X_2021']

In [42]:
panel_data.head()

,Jugador,Salario_2011,X_2011,Salario_2012,X_2012,Salario_2013,X_2013,Salario_2014,X_2014,Salario_2015,...,Salario_2017,X_2017,Salario_2018,X_2018,Salario_2019,X_2019,Salario_2020,X_2020,Salario_2021,X_2021
0,A.J. Burnett,NaN,NaN,NaN,NaN,NaN,NaN,1.287675e+07,0.633371,9718050.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A.J. Ellis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2763750.0,0.661257,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A.J. Griffin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A.J. Pierzynski,NaN,NaN,NaN,NaN,8724000.0,0.73127,1.917651e+05,-0.699237,2743920.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A.J. Pollock,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4239600.0,0.772974,NaN,NaN,NaN,NaN


In [43]:
panel_data = panel_data[['Jugador', 'Salario_2011', 'Salario_2012', 'Salario_2013', 'Salario_2014', 'Salario_2015', 'Salario_2016', 
                         'Salario_2017', 'Salario_2018', 'Salario_2019', 'Salario_2020', 'Salario_2021',
                         'X_2011', 'X_2012', 'X_2013', 'X_2014', 'X_2015', 'X_2016', 'X_2017', 'X_2018', 'X_2019', 'X_2020', 'X_2021', ]]

In [44]:
panel_data.head()

,Jugador,Salario_2011,Salario_2012,Salario_2013,Salario_2014,Salario_2015,Salario_2016,Salario_2017,Salario_2018,Salario_2019,...,X_2012,X_2013,X_2014,X_2015,X_2016,X_2017,X_2018,X_2019,X_2020,X_2021
0,A.J. Burnett,NaN,NaN,NaN,1.287675e+07,9718050.0,NaN,NaN,NaN,NaN,...,NaN,NaN,0.633371,0.776199,NaN,NaN,NaN,NaN,NaN,NaN
1,A.J. Ellis,NaN,NaN,NaN,NaN,NaN,NaN,2763750.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.661257,NaN,NaN,NaN,NaN
2,A.J. Griffin,NaN,NaN,NaN,NaN,NaN,572967.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.768155,NaN,NaN,NaN,NaN,NaN
3,A.J. Pierzynski,NaN,NaN,8724000.0,1.917651e+05,2743920.0,3443450.0,NaN,NaN,NaN,...,NaN,0.73127,-0.699237,-0.523384,-0.539488,NaN,NaN,NaN,NaN,NaN
4,A.J. Pollock,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4239600.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.772974,NaN,NaN


Para solo quedarnos con los datos que necesitamos -para el modelo del artículo- tenemos que quitar a todos los jugadores que solo han jugado una temporada en el periodo de análisis.

In [45]:
maximo_nan = panel_data.shape[1] - (1 + 1*2) - 1
print("Máximo número de 'nan' que pueden haber en un renglón: " + str(maximo_nan))

Máximo número de 'nan' que pueden haber en un renglón: 19


In [46]:
all_players_names = np.unique(panel_data['Jugador'])
index_drop = array.array('i',[2])

In [47]:
for i in range(0,len(all_players_names)):
    row = panel_data.iloc[i]
    row_length = row.size
    k = 0
    
    for j in range(row_length):
        if pd.isna(row[j]):
            k = k + 1
    
    if k > maximo_nan:
        index_drop.insert(-1, i)

index_group = np.array(index_drop)
index_drop = index_drop[:-1]

Los jugadores que quitaremos de la lista son los siguientes:

In [48]:
len(index_drop)

365

Quitemos a dichos jugadores

In [49]:
panel_data.drop(index_drop, inplace = True)

In [50]:
panel_data.reset_index(drop = True, inplace = True)

In [51]:
# Para asegurar que se siga cumpliendo;
panel_data = panel_data.sort_values(by = 'Jugador', ascending = True)

In [52]:
print("Nos quedan " + str(panel_data.shape[0]) + " jugadores en la base de datos.")

Nos quedan 244 jugadores en la base de datos.


In [53]:
panel_data.tail()

,Jugador,Salario_2011,Salario_2012,Salario_2013,Salario_2014,Salario_2015,Salario_2016,Salario_2017,Salario_2018,Salario_2019,...,X_2012,X_2013,X_2014,X_2015,X_2016,X_2017,X_2018,X_2019,X_2020,X_2021
239,Yasmani Grandal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16958400.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.718102,0.763763,NaN
240,Yovani Gallardo,NaN,NaN,NaN,NaN,NaN,10161000.0,NaN,2480780.0,NaN,...,NaN,NaN,NaN,NaN,0.710033,NaN,-0.538407,NaN,NaN,NaN
241,Yusmeiro Petit,NaN,NaN,NaN,NaN,NaN,2822500.0,NaN,3775100.0,NaN,...,NaN,NaN,NaN,NaN,0.750665,NaN,0.773003,NaN,NaN,0.685343
242,Zach Duke,NaN,1062180.0,569968.0,972910.0,5144850.0,NaN,NaN,NaN,NaN,...,0.777778,0.717958,-0.657538,-0.618058,NaN,NaN,NaN,NaN,NaN,NaN
243,Zack Greinke,NaN,NaN,22100800.0,NaN,NaN,38386000.0,NaN,NaN,NaN,...,NaN,-0.673092,NaN,NaN,-0.557441,NaN,NaN,NaN,NaN,NaN


Puesto que tendremos el mismo orden para los sueldo en valor presente 2021 y para las variables X, trabajemos por separado con los salarios.

In [188]:
adjusted_salary = panel_data.iloc[:,0:12]
year_x = panel_data.iloc[:,12:24]
year_x['Jugador'] = panel_data['Jugador']

In [147]:
adjusted_salary.head()

,Jugador,Salario_2011,Salario_2012,Salario_2013,Salario_2014,Salario_2015,Salario_2016,Salario_2017,Salario_2018,Salario_2019,Salario_2020,Salario_2021
0,A.J. Burnett,NaN,NaN,NaN,1.287675e+07,9718050.0,NaN,NaN,NaN,NaN,NaN,NaN
1,A.J. Pierzynski,NaN,NaN,8724000.0,1.917651e+05,2743920.0,3443450.0,NaN,NaN,NaN,NaN,NaN
2,Aaron Harang,NaN,3540600.0,NaN,NaN,5716500.0,NaN,NaN,NaN,NaN,NaN,NaN
3,Adam LaRoche,NaN,NaN,11632000.0,NaN,13719600.0,NaN,NaN,NaN,NaN,NaN,NaN
4,Adam Rosales,NaN,NaN,NaN,8.584500e+05,NaN,NaN,1381875.0,NaN,NaN,NaN,NaN


In [148]:
adjusted_salary.shape

(244, 12)

In [189]:
year_x.head()

,X_2011,X_2012,X_2013,X_2014,X_2015,X_2016,X_2017,X_2018,X_2019,X_2020,X_2021,Jugador
0,NaN,NaN,NaN,0.633371,0.776199,NaN,NaN,NaN,NaN,NaN,NaN,A.J. Burnett
1,NaN,NaN,0.73127,-0.699237,-0.523384,-0.539488,NaN,NaN,NaN,NaN,NaN,A.J. Pierzynski
2,NaN,-0.629194,NaN,NaN,-0.541273,NaN,NaN,NaN,NaN,NaN,NaN,Aaron Harang
3,NaN,NaN,0.68254,NaN,-0.618058,NaN,NaN,NaN,NaN,NaN,NaN,Adam LaRoche
4,NaN,NaN,NaN,-0.538407,NaN,NaN,0.757677,NaN,NaN,NaN,NaN,Adam Rosales


Recorramos todos los valores del salario a la izquierda para calcular $Y$ más adelante.

In [190]:
for i in range(0,len(adjusted_salary['Jugador'])):
    row = adjusted_salary.iloc[i]
    row_length = row.size
    sustitute = []
    
    for j in range(row_length):
        if pd.isna(row[j]) != True:
            sustitute.append(row[j])
            
    sustitute_length = len(sustitute)
    
    for k in range(row_length-sustitute_length):
        sustitute.append(np.nan)
        
    for j in range(row_length):
        adjusted_salary.iloc[i,j] = sustitute[j]

adjusted_salary.columns = ['Jugador', 'w_1', 'w_2', 'w_3', 'w_4', 'w_5', 'w_6', 
                           'w_7', 'w_8', 'w_9', 'w_10', 'w_11']

In [191]:
adjusted_salary.head()

,Jugador,w_1,w_2,w_3,w_4,w_5,w_6,w_7,w_8,w_9,w_10,w_11
0,A.J. Burnett,12876750.0,9.718050e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A.J. Pierzynski,8724000.0,1.917651e+05,2743920.0,3443450.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Aaron Harang,3540600.0,5.716500e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Adam LaRoche,11632000.0,1.371960e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Adam Rosales,858450.0,1.381875e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [192]:
adjusted_salary.tail()

,Jugador,w_1,w_2,w_3,w_4,w_5,w_6,w_7,w_8,w_9,w_10,w_11
239,Yasmani Grandal,16958400.0,19107750.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
240,Yovani Gallardo,10161000.0,2480780.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
241,Yusmeiro Petit,2822500.0,3775100.0,3030000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
242,Zach Duke,1062180.0,569968.0,972910.0,5144850.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
243,Zack Greinke,22100800.0,38386000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [193]:
# Creemos una copia
df_y = adjusted_salary.copy()
y_names = df_y['Jugador']

In [194]:
for i in range(0,len(y_names)):
    row = df_y.iloc[i]
    row_length = row.size
    k = 0

    sustitute = []

    for j in range(0,row_length):
        if pd.isna(row[j]) != True:
            k = k + 1

    sustitute.append(y_names[i])
    for j in range(1,k - 1):
        sustitute.append(row[j + 1]**0.5 - row[j]**0.5)

    sustitute_length = len(sustitute)

    for k in range(row_length - sustitute_length):
        sustitute.append(np.nan)

    for j in range(row_length):
        df_y.iloc[i,j] = sustitute[j]

df_y.columns = ['Jugador', 'Y_1', 'Y_2', 'Y_3', 'Y_4', 'Y_5', 'Y_6', 
                           'Y_7', 'Y_8', 'Y_9', 'Y_10', 'Y_11']

In [195]:
df_y.head()

,Jugador,y_1,y_2,y_3,y_4,y_5,y_6,y_7,y_8,y_9,y_10,y_11
0,A.J. Burnett,-471.040165,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A.J. Pierzynski,-2515.731853,1218.568225,199.175332,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Aaron Harang,509.272111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Adam LaRoche,293.426041,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Adam Rosales,249.006026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Repitamos parcialmente el proceso para la varibale $X$. Sin embargo, quitaremos el último valor de $X$ puesto que no lo necesitamos debido a cómo se calcula $Y$.

In [200]:
year_x = year_x[['Jugador', 'X_2011', 'X_2012', 'X_2013', 'X_2014', 'X_2015', 
                 'X_2016', 'X_2017', 'X_2018', 'X_2019', 'X_2020', 'X_2021']]
year_x.head()

,Jugador,X_2011,X_2012,X_2013,X_2014,X_2015,X_2016,X_2017,X_2018,X_2019,X_2020,X_2021
0,A.J. Burnett,NaN,NaN,NaN,0.633371,0.776199,NaN,NaN,NaN,NaN,NaN,NaN
1,A.J. Pierzynski,NaN,NaN,0.73127,-0.699237,-0.523384,-0.539488,NaN,NaN,NaN,NaN,NaN
2,Aaron Harang,NaN,-0.629194,NaN,NaN,-0.541273,NaN,NaN,NaN,NaN,NaN,NaN
3,Adam LaRoche,NaN,NaN,0.68254,NaN,-0.618058,NaN,NaN,NaN,NaN,NaN,NaN
4,Adam Rosales,NaN,NaN,NaN,-0.538407,NaN,NaN,0.757677,NaN,NaN,NaN,NaN


In [201]:
df_x = year_x.copy()
x_names = year_x['Jugador']

In [202]:
for i in range(0,len(x_names)):
    row = df_x.iloc[i]
    row_length = row.size
    sustitute = []
    
    for j in range(row_length):
        if pd.isna(row[j]) != True:
            sustitute.append(row[j])
            
    sustitute_length = len(sustitute)
    
    for k in range(row_length-sustitute_length):
        sustitute.append(np.nan)
        
    sustitute[sustitute_length - 1] = np.nan
    
    for j in range(row_length):
        df_x.iloc[i,j] = sustitute[j]

df_x.columns = ['Jugador', 'X_1', 'X_2', 'X_3', 'X_4', 'X_5', 'X_6', 
                           'X_7', 'X_8', 'X_9', 'X_10', 'X_11']

In [203]:
df_x.head()

,Jugador,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11
0,A.J. Burnett,0.633371,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A.J. Pierzynski,0.731270,-0.699237,-0.523384,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Aaron Harang,-0.629194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Adam LaRoche,0.682540,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Adam Rosales,-0.538407,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


La estrategia para construir la base de datos para la regresión consiste en el siguiente proceso:
- Crear dataframes que contengan la columna del nombre del jugador y la variable de $X$ y $Y$ del mismo subíndice.
- Hacer merge de las bases de datos que contengan el mismo subíndice.
- Concatenar todas las bases de datos.
- Eliminar las filas que contengan datos faltantes.

In [223]:
regression_list = [None]*(period-1)

In [224]:
for i in range(0,len(regression_list)):
    merge = pd.merge(df_y.iloc[:,[0,i+1]], df_x.iloc[:,[0,i+1]])
    regression_list[i] = merge
    regression_list[i].columns = ['Jugador', 'Y', 'X']

In [233]:
df_article_regression = regression_list[0]

for i in range(1,len(regression_list)):
    df_article_regression = pd.concat([df_article_regression, regression_list[i]])

In [239]:
df_article_regression = df_article_regression.sort_values(by = 'Jugador', ascending = True)
df_article_regression.reset_index(drop = True, inplace = True)
df_article_regression

,Jugador,Y,X
0,A.J. Burnett,-471.040165,0.633371
1,A.J. Burnett,NaN,NaN
2,A.J. Burnett,NaN,NaN
3,A.J. Burnett,NaN,NaN
4,A.J. Burnett,NaN,NaN
...,...,...,...
2435,Zack Greinke,NaN,NaN
2436,Zack Greinke,NaN,NaN
2437,Zack Greinke,NaN,NaN
2438,Zack Greinke,1494.494835,-0.673092


Por último, eliminemos las filas que contengan filas con datos *NaN*.

In [240]:
df_article_regression.dropna(inplace = True)
df_article_regression.reset_index(drop = True, inplace = True)
df_article_regression

,Jugador,Y,X
0,A.J. Burnett,-471.040165,0.633371
1,A.J. Pierzynski,-2515.731853,0.731270
2,A.J. Pierzynski,1218.568225,-0.699237
3,A.J. Pierzynski,199.175332,-0.523384
4,Aaron Harang,509.272111,-0.629194
...,...,...,...
358,Yusmeiro Petit,262.931895,0.750665
359,Zach Duke,-275.658921,0.777778
360,Zach Duke,1281.864177,-0.657538
361,Zach Duke,231.399752,0.717958


Observemos que ya no hay valores *NaN*.

In [241]:
pd.isnull(df_article_regression).sum(axis=0)

Jugador    0
Y          0
X          0
dtype: int64

Finalmente, exportemos la nueva base de datos para que se pueda hacer el análisis en cualquier lenguaje

In [242]:
df_article_regression.to_csv('Data/New_Data/Models/Article/article_regression.csv', index = False)